In [ ]:
from TTS.utils.io import load_fsspec
import torch
def patched_load_fsspec(path, map_location=None, cache=True, **kwargs):
    if cache:
        return torch.load(path, map_location=map_location, weights_only=False, **kwargs)
    else:
        import fsspec
        with fsspec.open(path, "rb") as f:
            return torch.load(f, map_location=map_location, weights_only=False, **kwargs)
import TTS.utils.io
TTS.utils.io.load_fsspec = patched_load_fsspec

from TTS.api import TTS
tts = TTS(model_name="tts_models/multilingual/multi-dataset/xtts_v2", progress_bar=True, gpu=False)
print(tts.speakers)
tts.tts_to_file(text="¡Hola, esto es una prueba!", speaker="female-en-4", language="es", file_path="test_xtts.wav")

In [ ]:
from TTS.api import TTS
tts = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC", progress_bar=True, gpu=False)
tts.tts_to_file(text="Hello, this is a test.", file_path="test_tacotron.wav")

In [ ]:
import sys
print(sys.executable)

In [ ]:
pip show torch TTS

In [3]:
import os
import sys
import torch
from pydub import AudioSegment
import logging
import traceback

# Set up logging first
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# Patch load_fsspec to disable weights_only for xtts_v2
from TTS.utils.io import load_fsspec
def patched_load_fsspec(path, map_location=None, cache=True, **kwargs):
    if cache:
        return torch.load(path, map_location=map_location, weights_only=False, **kwargs)
    else:
        import fsspec
        with fsspec.open(path, "rb") as f:
            return torch.load(f, map_location=map_location, weights_only=False, **kwargs)
import TTS.utils.io
TTS.utils.io.load_fsspec = patched_load_fsspec

# Verify TTS import
try:
    from TTS.api import TTS
    logger.info(f"TTS class type: {type(TTS)}")
except ImportError as e:
    print(f"Failed to import TTS: {e}")
    sys.exit(1)

# Create output directory
os.makedirs("audios", exist_ok=True)

# List of characters and their lines
dialogo = [
    ("MÍA", "¡Chicos, chicos! ¡Lo vi otra vez! El glitch ese... apareció en el comedor. Una figura rara, como un fantasma digital, y luego puf, desapareció."),
    ("ZOE", "Mía, si sigues viendo fantasmas en la red AR, tal vez deberías revisar tus lentes. O dejar de tomar tanto café."),
    ("KAI", "Déjala, Zoe. Mía tiene razón. Yo también vi algo raro ayer en mi simulación. Era como... una sombra que no debería estar ahí."),
    ("ZOE", "Espera, ¿tú también? Okay, esto ya no es una broma. Si hay un glitch en la red AR del campus, podría colapsar todo: exámenes, simulaciones, hasta los drones de pizza."),
    ("MÍA", "¿Y si es un hacker? O... no sé, ¿una IA rebelde? ¡Leí que eso pasa en las pelis!"),
    ("KAI", "Mía, relaja, no estamos en Matrix. Aunque... un misterio así sería épico para mi próximo proyecto AR. Imagina: El Campus Embrujado."),
    ("ZOE", "Tú y tus proyectos. Necesitamos encontrar el origen de este glitch antes de que el Profesor Vega nos suspenda por alterar la red."),
    ("PROFESOR VEGA", "¿Alterar la red, dicen? Interesante. Zoe, Kai, Mía, ¿por qué tengo reportes de un fantasma digital interrumpiendo las simulaciones del campus?"),
    ("MÍA", "¡No fuimos nosotros, lo juro! Solo... lo vimos."),
    ("ZOE", "Profesor, si hay un glitch, puedo rastrearlo. Pero necesito acceso al núcleo de la red AR."),
    ("PROFESOR VEGA", "¿Acceso al núcleo? Eso es como darle las llaves de una nave estelar a un cadete. Pero... me intrigan. Les doy 24 horas para encontrar ese glitch. Si no, limpiarán servidores por un mes."),
    ("KAI", "Eso suena a castigo medieval."),
    ("MÍA", "¿Y si el glitch no es un error? ¿Y si es... algo vivo?"),
    ("ZOE", "Sea lo que sea, lo voy a cazar. Kai, prepara un entorno AR para atraparlo. Mía, tú... intenta no romper nada."),
    ("MÍA", "¡Oye, soy útil! Puedo... grabar todo para el vlog del campus."),
    ("KAI", "Eso, Mía, haznos virales mientras salvamos el campus.")
]

# Assign voices to characters
personajes_voces = {
    "MÍA": "tts_models/multilingual/multi-dataset/xtts_v2",
    "ZOE": "tts_models/multilingual/multi-dataset/xtts_v2",
    "KAI": "tts_models/multilingual/multi-dataset/xtts_v2",
    "PROFESOR VEGA": "tts_models/multilingual/multi-dataset/xtts_v2"
}

# Placeholder speaker IDs (update after checking tts.speakers)
voces_ids = {
    "MÍA": "female-en-4",
    "ZOE": "female-en-3",
    "KAI": "male-en-3",
    "PROFESOR VEGA": "male-en-1"
}

# Fallback model
FALLBACK_MODEL = "tts_models/en/ljspeech/tacotron2-DDC"

def initialize_tts_model(model_name, gpu=False):
    """Initialize TTS model with error handling and fallback."""
    try:
        logger.info(f"Attempting to load model: {model_name}")
        tts = TTS(model_name=model_name, progress_bar=True, gpu=gpu)
        logger.info(f"Successfully loaded model: {model_name}")
        # Log available speakers and multilingual support
        if hasattr(tts, "is_multi_speaker") and tts.is_multi_speaker:
            logger.info(f"Available speakers: {tts.speakers}")
        if hasattr(tts, "is_multi_lingual") and tts.is_multi_lingual:
            logger.info("Model supports multilingual input")
        else:
            logger.info("Model is not multilingual")
        return tts
    except Exception as e:
        logger.error(f"Failed to load model {model_name}: {str(e)}")
        logger.error(traceback.format_exc())
        logger.info(f"Falling back to model: {FALLBACK_MODEL}")
        try:
            tts = TTS(model_name=FALLBACK_MODEL, progress_bar=True, gpu=gpu)
            logger.warning(f"Using fallback model {FALLBACK_MODEL}, which does not support multi-speaker or multilingual voices. All characters will use the default English voice.")
            return tts
        except Exception as e:
            logger.error(f"Failed to load fallback model {FALLBACK_MODEL}: {str(e)}")
            raise RuntimeError("Could not initialize any TTS model.")

# Initialize the model
try:
    tts = initialize_tts_model(personajes_voces["MÍA"], gpu=False)
except RuntimeError as e:
    logger.error(str(e))
    raise

# Generate audio for each line
audio_parts = []
for i, (personaje, texto) in enumerate(dialogo):
    output_file = f"audios/{i:02d}_{personaje}.wav"
    try:
        logger.info(f"Generating audio for {personaje}: {texto[:50]}...")
        if hasattr(tts, "is_multi_speaker") and tts.is_multi_speaker:
            if voces_ids[personaje] in tts.speakers:
                # Use language only for multilingual models
                if hasattr(tts, "is_multi_lingual") and tts.is_multi_lingual:
                    tts.tts_to_file(text=texto, speaker=voces_ids[personaje], language="es", file_path=output_file)
                else:
                    tts.tts_to_file(text=texto, speaker=voces_ids[personaje], file_path=output_file)
            else:
                logger.warning(f"Speaker {voces_ids[personaje]} not found for {personaje}. Using default voice.")
                if hasattr(tts, "is_multi_lingual") and tts.is_multi_lingual:
                    tts.tts_to_file(text=texto, language="es", file_path=output_file)
                else:
                    tts.tts_to_file(text=texto, file_path=output_file)
        else:
            logger.warning(f"Model does not support multi-speaker. Using default voice for {personaje}.")
            if hasattr(tts, "is_multi_lingual") and tts.is_multi_lingual:
                tts.tts_to_file(text=texto, language="es", file_path=output_file)
            else:
                tts.tts_to_file(text=texto, file_path=output_file)
        audio_parts.append(AudioSegment.from_wav(output_file))
        logger.info(f"Generated audio: {output_file}")
    except Exception as e:
        logger.error(f"Failed to generate audio for {personaje}: {str(e)}")
        continue

# Combine all audio parts
if audio_parts:
    narracion_completa = sum(audio_parts)
    output_final = "audios/escena_completa.wav"
    narracion_completa.export(output_final, format="wav")
    logger.info(f"✅ Archivo final generado: {output_final}")
else:
    logger.error("No audio parts were generated. Check logs for errors.")

2025-05-12 00:14:16,033 - INFO - TTS class type: <class 'type'>
2025-05-12 00:14:16,035 - INFO - Attempting to load model: tts_models/multilingual/multi-dataset/xtts_v2


 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts


2025-05-12 00:14:37,277 - INFO - Successfully loaded model: tts_models/multilingual/multi-dataset/xtts_v2
2025-05-12 00:14:37,279 - ERROR - Failed to load model tts_models/multilingual/multi-dataset/xtts_v2: 'TTS' object has no attribute 'speakers'
2025-05-12 00:14:37,281 - ERROR - Traceback (most recent call last):
  File "C:\Users\Administrador\AppData\Local\Temp\ipykernel_1820\2468024688.py", line 82, in initialize_tts_model
    logger.info(f"Available speakers: {tts.speakers}")
                                       ^^^^^^^^^^^^
  File "C:\Users\Administrador\tts_env\Lib\site-packages\torch\nn\modules\module.py", line 1940, in __getattr__
    raise AttributeError(
AttributeError: 'TTS' object has no attribute 'speakers'

2025-05-12 00:14:37,282 - INFO - Falling back to model: tts_models/en/ljspeech/tacotron2-DDC


 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio P

2025-05-12 00:14:38,471 - WARNING - Using fallback model tts_models/en/ljspeech/tacotron2-DDC, which does not support multi-speaker or multilingual voices. All characters will use the default English voice.
2025-05-12 00:14:38,642 - INFO - Generating audio for MÍA: ¡Chicos, chicos! ¡Lo vi otra vez! El glitch ese......
2025-05-12 00:14:38,642 - WARNING - Model does not support multi-speaker. Using default voice for MÍA.


...

 > Processing time: 5.63813042640686
 > Real-time factor: 0.5254648335627209
 > Text splitted to sentences.
['¡Oye, soy útil!', 'Puedo... grabar todo para el vlog del campus.']


2025-05-12 00:16:03,920 - INFO - Generated audio: audios/14_MÍA.wav
2025-05-12 00:16:03,920 - INFO - Generating audio for KAI: Eso, Mía, haznos virales mientras salvamos el camp...
2025-05-12 00:16:03,922 - WARNING - Model does not support multi-speaker. Using default voice for KAI.


 > Processing time: 3.333513021469116
 > Real-time factor: 0.49323573466954324
 > Text splitted to sentences.
['Eso, Mía, haznos virales mientras salvamos el campus.']


2025-05-12 00:16:06,908 - INFO - Generated audio: audios/15_KAI.wav
2025-05-12 00:16:06,924 - INFO - ✅ Archivo final generado: audios/escena_completa.wav


 > Processing time: 2.943063974380493
 > Real-time factor: 0.4893123464462682


In [5]:
import os
from gtts import gTTS
from pydub import AudioSegment
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# Create output directory
os.makedirs("audios", exist_ok=True)

# List of characters and their lines
dialogo = [
    ("MÍA", "¡Chicos, chicos! ¡Lo vi otra vez! El glitch ese... apareció en el comedor. Una figura rara, como un fantasma digital, y luego puf, desapareció."),
    ("ZOE", "Mía, si sigues viendo fantasmas en la red AR, tal vez deberías revisar tus lentes. O dejar de tomar tanto café."),
    ("KAI", "Déjala, Zoe. Mía tiene razón. Yo también vi algo raro ayer en mi simulación. Era como... una sombra que no debería estar ahí."),
    ("ZOE", "Espera, ¿tú también? Okay, esto ya no es una broma. Si hay un glitch en la red AR del campus, podría colapsar todo: exámenes, simulaciones, hasta los drones de pizza."),
    ("MÍA", "¿Y si es un hacker? O... no sé, ¿una IA rebelde? ¡Leí que eso pasa en las pelis!"),
    ("KAI", "Mía, relaja, no estamos en Matrix. Aunque... un misterio así sería épico para mi próximo proyecto AR. Imagina: El Campus Embrujado."),
    ("ZOE", "Tú y tus proyectos. Necesitamos encontrar el origen de este glitch antes de que el Profesor Vega nos suspenda por alterar la red."),
    ("PROFESOR VEGA", "¿Alterar la red, dicen? Interesante. Zoe, Kai, Mía, ¿por qué tengo reportes de un fantasma digital interrumpiendo las simulaciones del campus?"),
    ("MÍA", "¡No fuimos nosotros, lo juro! Solo... lo vimos."),
    ("ZOE", "Profesor, si hay un glitch, puedo rastrearlo. Pero necesito acceso al núcleo de la red AR."),
    ("PROFESOR VEGA", "¿Acceso al núcleo? Eso es como darle las llaves de una nave estelar a un cadete. Pero... me intrigan. Les doy 24 horas para encontrar ese glitch. Si no, limpiarán servidores por un mes."),
    ("KAI", "Eso suena a castigo medieval."),
    ("MÍA", "¿Y si el glitch no es un error? ¿Y si es... algo vivo?"),
    ("ZOE", "Sea lo que sea, lo voy a cazar. Kai, prepara un entorno AR para atraparlo. Mía, tú... intenta no romper nada."),
    ("MÍA", "¡Oye, soy útil! Puedo... grabar todo para el vlog del campus."),
    ("KAI", "Eso, Mía, haznos virales mientras salvamos el campus.")
]

# Voice variation settings (simulate different voices)
voice_settings = {
    "MÍA": {"slow": False, "pitch_shift": 2},  # Voz más aguda
    "ZOE": {"slow": False, "pitch_shift": -1},  # Voz ligeramente más grave
    "KAI": {"slow": True, "pitch_shift": -4},   # Voz más lenta y grave
    "PROFESOR VEGA": {"slow": True, "pitch_shift": -6}  # Voz muy grave
}

# Generate audio for each line
audio_parts = []
for i, (personaje, texto) in enumerate(dialogo):
    output_file = f"audios/{i:02d}_{personaje}.mp3"
    try:
        logger.info(f"Generating audio for {personaje}: {texto[:50]}...")
        tts = gTTS(text=texto, lang="es", slow=voice_settings[personaje]["slow"])
        tts.save(output_file)
        # Convert MP3 to WAV and apply pitch shift
        audio = AudioSegment.from_mp3(output_file)
        # Apply pitch shift (semitones)
        pitch_shifted = audio._spawn(audio.raw_data, overrides={
            "frame_rate": int(audio.frame_rate * (2.0 ** (voice_settings[personaje]["pitch_shift"] / 12.0)))
        })
        wav_file = output_file.replace(".mp3", ".wav")
        pitch_shifted.export(wav_file, format="wav")
        audio_parts.append(AudioSegment.from_wav(wav_file))
        logger.info(f"Generated audio: {wav_file}")
    except Exception as e:
        logger.error(f"Failed to generate audio for {personaje}: {str(e)}")
        continue

# Combine all audio parts
if audio_parts:
    narracion_completa = sum(audio_parts)
    output_final = "audios/escena_completa.wav"
    narracion_completa.export(output_final, format="wav")
    logger.info(f"✅ Archivo final generado: {output_final}")
else:
    logger.error("No audio parts were generated. Check logs for errors.")

2025-05-12 00:20:36,401 - INFO - Generating audio for MÍA: ¡Chicos, chicos! ¡Lo vi otra vez! El glitch ese......
2025-05-12 00:20:49,810 - INFO - Generated audio: audios/00_MÍA.wav
2025-05-12 00:20:49,810 - INFO - Generating audio for ZOE: Mía, si sigues viendo fantasmas en la red AR, tal ...
2025-05-12 00:20:51,803 - INFO - Generated audio: audios/01_ZOE.wav
2025-05-12 00:20:51,804 - INFO - Generating audio for KAI: Déjala, Zoe. Mía tiene razón. Yo también vi algo r...
2025-05-12 00:20:54,887 - INFO - Generated audio: audios/02_KAI.wav
2025-05-12 00:20:54,887 - INFO - Generating audio for ZOE: Espera, ¿tú también? Okay, esto ya no es una broma...
2025-05-12 00:20:59,003 - INFO - Generated audio: audios/03_ZOE.wav
2025-05-12 00:20:59,003 - INFO - Generating audio for MÍA: ¿Y si es un hacker? O... no sé, ¿una IA rebelde? ¡...
2025-05-12 00:20:59,659 - INFO - Generated audio: audios/04_MÍA.wav
2025-05-12 00:20:59,659 - INFO - Generating audio for KAI: Mía, relaja, no estamos en Matrix. A